In [1]:
import pandas as pd
import re
from itertools import groupby, chain

In [2]:
import plotly
import plotly.graph_objects as go

In [3]:
def get_traces(filename, value, color):
    f = open(filename, "r")
    lines = f.readlines()

    lines = list(map(lambda x: x.strip(), lines))

    def is_splitting_row(x):
        return re.match('^2\s+.+', x) is not None

    split_list = [list(group) for k, group in groupby(lines, is_splitting_row) if not k]

    split_list = split_list[1:]

    def extract(x):
        try:
            val = float(re.match('.+\s+([^\s]+)', x[-2]).groups()[0])
        except:
            val = 0

        points = ' '.join(x[:-2]).split()
        filtered_points = [elem for idx, elem in enumerate(points) if (idx + 1) % 3 != 0]
        point_tuples = list(zip(filtered_points[::2], filtered_points[1::2]))
        return dict(val = val, points = point_tuples)    

    result = list(map(extract, split_list))

    test_data = [x for x in result if x['val'] == value]

    traces = []

    for item in test_data:
        trace = go.Scatter(x=[i[0] for i in item['points']], 
                           y=[i[1] for i in item['points']], 
                           mode='lines',  
                           line=dict(color=color),
                           name = filename + ' - ' + str(value),
                           showlegend  = False)
        traces.append(trace)

    traces.append(go.Scatter(x=[None], y=[None], mode='markers',
                       marker=dict(size=10, color=color), showlegend=True, name=filename + ' - ' + str(value)))
    return traces

In [4]:
def plot(config):
    traces = []

    for item in config:    
        traces = traces + get_traces(item['filename'], item['value'], item['color'])

    fig = go.Figure(data=traces)
    fig.update_xaxes(range=[500, 1200], title='Temperature')
    fig.update_yaxes(range=[4000, 12000], title='Pressure')
    fig.update_layout(
        autosize=False,
        width=900,
        height=700)

    fig.show()

In [7]:
config = [
    {
        'filename': "data/geology/Grt-Alm_domplt.plt",
        'value': 0.5,
        'color':'red'
    },{
        'filename': "data/geology/Grt-Gr_domplt.plt",
        'value': 0.2,
        'color':'blue'
    },{
        'filename': "data/geology/Grt-Py_domplt.plt",
        'value': 0.3,
        'color':'green'
    },{
        'filename': "data/geology/Grt-Py_domplt.plt",
        'value': 0,
        'color':'gray'
    }
]
plot(config)